Преподаватель: Алексей Кузьмин, Алексей Миронов

Задание 1.  
Обучите нейронную сеть решать шифр цезаря.  
Что надо сделать:  
1.Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)  
2.Сделать нейронную сеть  
3.Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)  
4.Проверить качество  

In [46]:
import random
import torch
import time

# 1.Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)

In [26]:
key = 13
vocab = [char for char in 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ']


def encrypt(text):
    indexes = [vocab.index(char) for char in text]
    encrypted_indexes = [(idx + key) % len(vocab) for idx in indexes]
    encrypted_chars = [vocab[idx] for idx in encrypted_indexes]
    encrypted = ''.join(encrypted_chars)
    return encrypted

In [27]:
print(encrypt('АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'))

МНОПРСТУФХЦЧШЩЪЫЬЭЮЯАБВГДЕЁЖЗИЙКЛ


In [28]:
num_examples = 128
message_length = 32


def dataset(num_examples):
    dataset = []
    for x in range(num_examples):
        ex_out = ''.join([random.choice(vocab) for x in range(message_length)])
        ex_in = encrypt(''.join(ex_out))
        ex_in = [vocab.index(x) for x in ex_in]
        ex_out = [vocab.index(x) for x in ex_out]
        dataset.append([torch.tensor(ex_in), torch.tensor(ex_out)])
    return dataset

2.Сделать нейронную сеть

# LSTM

In [39]:
embedding_dim = 10
hidden_dim = 10
vocab_size = len(vocab)

embed = torch.nn.Embedding(vocab_size, embedding_dim)
lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
linear = torch.nn.Linear(hidden_dim, vocab_size)
softmax = torch.nn.functional.softmax
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(embed.parameters()) +
                             list(lstm.parameters()) +
                             list(linear.parameters()), lr=0.001)

In [40]:
def zero_hidden():
    return (torch.zeros(1, 1, hidden_dim),
            torch.zeros(1, 1, hidden_dim))

3.Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)

In [41]:
num_epochs = 10

accuracies, max_accuracy = [], 0
for x in range(num_epochs):
    print('Epoch: {}'.format(x))
    for encrypted, original in dataset(num_examples):
        lstm_in = embed(encrypted)
        lstm_in = lstm_in.unsqueeze(1)
        lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
        scores = linear(lstm_out)
        scores = scores.transpose(1, 2)
        original = original.unsqueeze(1)
        loss = loss_fn(scores, original) 
        loss.backward()
        optimizer.step()
    print('Loss: {:6.4f}'.format(loss.item()))

Epoch: 0
Loss: 3.1263
Epoch: 1
Loss: 2.2448
Epoch: 2
Loss: 1.1692
Epoch: 3
Loss: 0.6322
Epoch: 4
Loss: 0.5210
Epoch: 5
Loss: 0.2631
Epoch: 6
Loss: 0.1590
Epoch: 7
Loss: 0.0738
Epoch: 8
Loss: 0.0579
Epoch: 9
Loss: 0.0274


4.Проверить качество 

In [35]:
with torch.no_grad():
        matches, total = 0, 0
        for encrypted, original in dataset(num_examples):
            lstm_in = embed(encrypted)
            lstm_in = lstm_in.unsqueeze(1)
            lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
            scores = linear(lstm_out)
            predictions = softmax(scores, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, original).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 100.00%


# RNN

In [ ]:
embedding_dim = 10
hidden_dim = 10
vocab_size = len(vocab)

In [42]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, sentences, state=None):
        ## Здесь применить
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [43]:
model = Network()

In [44]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [50]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for encrypted, original in dataset(num_examples):
        
        optimizer.zero_grad()
        answers = model.forward(encrypted.unsqueeze(1))
        answers = answers.view(-1, vocab_size)
        loss = criterion(answers, original)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.131, Train loss: 3.231
Epoch 1. Time: 0.123, Train loss: 2.597
Epoch 2. Time: 0.125, Train loss: 2.016
Epoch 3. Time: 0.128, Train loss: 1.531
Epoch 4. Time: 0.131, Train loss: 1.135
Epoch 5. Time: 0.125, Train loss: 0.868
Epoch 6. Time: 0.124, Train loss: 0.674
Epoch 7. Time: 0.123, Train loss: 0.539
Epoch 8. Time: 0.124, Train loss: 0.443
Epoch 9. Time: 0.127, Train loss: 0.361


In [59]:
with torch.no_grad():
        matches, total = 0, 0
        for encrypted, original in dataset(num_examples):
            answers = model.forward(encrypted.unsqueeze(1))
            predictions = torch.nn.functional.softmax(answers, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, original).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 100.00%


Задание 2.  
Выполнить практическую работу из лекционного ноутбука.  
а) построить RNN-ячейку на основе полносвязных слоев  
б) применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”  
  